In [1]:
from utils import *
import torch
import streamlit as st
from utils import *
from langchain.prompts import ChatPromptTemplate
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import logging
from importlib import reload


In [2]:
device = torch.device("cuda")
torch.cuda.is_available()
# torch.Tensor(1).to(device)

# reload(src/utils)

True

In [3]:
load_dotenv()  # take environment variables from .env.
index_name = "pubmedbert-sentence-transformer-400"
HUGGINGFACE_TOKEN = os.getenv('HUGGINGFACE_TOKEN')
HUGGINGFACE_USERNAME = os.getenv('HUGGINGFACE_USERNAME')
HUGGINGFACE_DATASET_NAME = os.getenv('HUGGINGFACE_DATASET_NAME')
ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')
QA_VALIDATION_DATASET = os.getenv('QA_VALIDATION_DATASET')
QA_VALIDATION_TOKEN = os.getenv('QA_VALIDATION_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
def _parse(text):
    return text.strip("**")


In [4]:

model_name = "NeuML/pubmedbert-base-embeddings"
device = 'cuda:0'
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs={'device': device}
)

elastic_vector_search = ElasticsearchStore(
        es_cloud_id = ELASTIC_CLOUD_ID,
        index_name = index_name,
        embedding = embeddings,
        es_api_key = ELASTIC_API_KEY
)

retriever = elastic_vector_search.as_retriever(search_kwargs={"k":3})


In [5]:
print(retriever.get_relevant_documents("what did he say about ketanji brown jackson"))

[Document(page_content='" until now, i have not responded to the undocumented and untrue portrayal of me in the hoffman report, or the newspaper and email accounts of my alleged role in \\ " torture. \\ " i have waited several years because i believed that the american psychological association leadership would completely disavow the hoffman report and find a way to restore my reputation as well as that of the other former american psychological association presidents and the small number of its senior staff who were also falsely accused. although this has not yet happened, i remain optimistic that it will. here i present the facts as they relate to me. "', metadata={'title': 'The American Psychological Association\'s Hoffman report allegations of my association with the "CIA torture" program.', 'publication_date': '03/09/2018', 'id': 11112, 'authors': ['Joseph D Matarazzo']}), Document(page_content='##y and know yourself, you need not fear the result of a hundred battles.\'"', metadat

In [19]:
query = "What were the most commonly used methods for bone age assessment in the studies reviewed?"

In [20]:
# rag = rag_pipeline(model_id, index_name)

## Rewrite the input text
rewrite_prompt = hub.pull("langchain-ai/rewrite")
rewrite_llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

rewriter = rewrite_prompt | rewrite_llm | StrOutputParser() | _parse
rewritten_input_text = rewriter.invoke({"x": query})

logging.info(f"Original input: {query}. Transformed input: {rewritten_input_text}")

print(rewritten_input_text)

Most commonly used methods for bone age assessment in medical studies**.


In [26]:
docs = retriever.get_relevant_documents(rewritten_input_text)

for doc in docs:
    print(doc.page_content)

" the accuracy and consistency of bone age assessments ( baa ) using standard methods can vary with physicians'level of experience. "
" bone age is one of biological indicators of maturity used in clinical practice and it is a very important parameter of a child \ u2019s assessment, especially in paediatric endocrinology. the most widely used method of bone age assessment is by performing a hand and wrist radiograph and its analysis with greulich - pyle or tanner - whitehouse atlases, although it has been about 60 years since they were published. due to the progress in the area of computer - aided diagnosis and application of artificial intelligence in medicine, lately, numerous programs for automatic bone age assessment have been created. most of them have been verified in clinical studies in comparison to traditional methods, showing good precision while eliminating inter - and intra - rater variability and significantly reducing the time of assessment. additionally, there are availa

In [21]:
model = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

ANSWER_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. Keep your answers shorter than 25 words.
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

ANSWER_PROMPT_medical= ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Assume that the reader has in depth knowledge into the medical field and is familiar with all the terms used, so they do not need to be explained. Keep your answers shorter than 25 words.
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT_medical
    | model
    | StrOutputParser()
)

ans = chain.invoke(rewritten_input_text)

print("---- Answer ----")
print(ans)

---- Answer ----
Hand and wrist radiograph analysis with Greulich-Pyle or Tanner-Whitehouse atlases is the most commonly used method for bone age assessment in medical studies.


In [ ]:
## The grand scientific challenge in materials research is to accelerate materials discovery and development for emerging and future technologies by integrating automation with artificial intelligence
# "Between 2019 Q4 and 2020 Q3 phenethyl 4 ANPP was detected in 25 1 054 fentanyl cases in the USA"

In [16]:
import json

with open("../data/evaluation_dataset.json", 'r') as file:
    json_data = file.read()

manual_inspection = json.loads(json_data)

for item in manual_inspection:
    ans = chain.invoke(rewriter.invoke({"x": item["question"]}))
    # print("Question: {}\nGround truth: {}\nGenerated answer: {}\n\n".format(item["question"], item["paragraph"], ans))
    item["generated_answer"] = ans


TypeError: dump() missing 1 required positional argument: 'fp'

In [18]:

with open("../data/evaluation_dataset_openai.json", 'w') as file:
    json.dump(manual_inspection,file, indent=2)